In [0]:
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
import tensorflow as tf

Using TensorFlow backend.


In [0]:
!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id':"1CQU5Jvf3X3FtbJZfvwt3BmSeit4Dq1Bo"})
download.GetContentFile('diabetes.csv')

In [0]:
df = pd.read_csv('diabetes.csv')

In [0]:
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Checking for any Null Values

In [0]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [0]:
input_layers = df.drop(['Outcome'],axis=1)
input_layers

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [0]:
output_pred = df["Outcome"]
output_pred

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

Converting the features into list

In [0]:
input_layers=input_layers.values
output_pred=output_pred.values

In [0]:
print(input_layers)

[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]


In [0]:
output_pred

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(input_layers,output_pred,test_size=.3,random_state=0)

In [0]:
print(X_train)
print(X_test)

[[0.00e+00 1.51e+02 9.00e+01 ... 4.21e+01 3.71e-01 2.10e+01]
 [1.00e+00 8.30e+01 6.80e+01 ... 1.82e+01 6.24e-01 2.70e+01]
 [2.00e+00 1.22e+02 7.00e+01 ... 3.68e+01 3.40e-01 2.70e+01]
 ...
 [4.00e+00 9.40e+01 6.50e+01 ... 2.47e+01 1.48e-01 2.10e+01]
 [1.10e+01 8.50e+01 7.40e+01 ... 3.01e+01 3.00e-01 3.50e+01]
 [5.00e+00 1.36e+02 8.20e+01 ... 0.00e+00 6.40e-01 6.90e+01]]
[[1.000e+00 1.990e+02 7.600e+01 ... 4.290e+01 1.394e+00 2.200e+01]
 [2.000e+00 1.070e+02 7.400e+01 ... 3.360e+01 4.040e-01 2.300e+01]
 [4.000e+00 7.600e+01 6.200e+01 ... 3.400e+01 3.910e-01 2.500e+01]
 ...
 [1.000e+00 9.500e+01 6.000e+01 ... 2.390e+01 2.600e-01 2.200e+01]
 [1.000e+00 1.060e+02 7.600e+01 ... 3.750e+01 1.970e-01 2.600e+01]
 [1.000e+01 1.220e+02 7.800e+01 ... 2.760e+01 5.120e-01 4.500e+01]]


In [0]:
from keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten
from keras.optimizers import Adam,sgd
from sklearn.metrics import mean_absolute_error
from keras import layers, metrics
import keras

In [0]:
!pip3 install 'gast==0.2.2'
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [0]:
def model(X_train,X_test,y_train,y_test):
    logi = Sequential()
    logi.add(Dense(300, activation="relu",input_shape=(8,)))
    logi.add(Dropout(0.4))
    logi.add(Dense(10, activation="relu"))
    logi.add(Dropout(0.1))
    logi.add(Dense(4, activation="relu"))
    logi.add(Dense(1,activation='sigmoid'))
    logi.compile(loss='binary_crossentropy', optimizer=sgd(), metrics=['accuracy'])
    logi.fit(X_train, y_train, epochs=100)
    a = logi.evaluate(X_test, y_test)
    return a

In [0]:
model(X_train,X_test,y_train,y_test)

Epoch 1/100
537/537 [==============================] - 0s 288us/step - loss: 0.8600 - accuracy: 0.4991
Epoch 2/100
537/537 [==============================] - 0s 59us/step - loss: 0.6892 - accuracy: 0.6220
Epoch 3/100
537/537 [==============================] - 0s 44us/step - loss: 0.7143 - accuracy: 0.6089
Epoch 4/100
537/537 [==============================] - 0s 45us/step - loss: 0.6741 - accuracy: 0.6387
Epoch 5/100
537/537 [==============================] - 0s 53us/step - loss: 0.6774 - accuracy: 0.6387
Epoch 6/100
537/537 [==============================] - 0s 48us/step - loss: 0.6791 - accuracy: 0.6387
Epoch 7/100
537/537 [==============================] - 0s 48us/step - loss: 0.6749 - accuracy: 0.6387
Epoch 8/100
537/537 [==============================] - 0s 50us/step - loss: 0.6692 - accuracy: 0.6387
Epoch 9/100
537/537 [==============================] - 0s 51us/step - loss: 0.6821 - accuracy: 0.6387
Epoch 10/100
537/537 [==============================] - 0s 52us/step - loss: 0.67

[0.6136281348410106, 0.6796537041664124]